In [ ]:
import glob
import re
import sys

import matplotlib.pyplot as plt
import numpy as np

basedir = "../"

sys.path.append(basedir)


from utils import configure_plotting, find_results_files, load_results, natural_sort

configure_plotting(basedir)

In [ ]:
default_figsize = plt.rcParams["figure.figsize"]

In [ ]:
path = "outdir_rerun/ins_nlive_gaussian_16d_nlive*"

In [ ]:
result_dirs = natural_sort(glob.glob(path))
regex = re.compile(r"\d+")
nlive_values = [int(regex.findall(p)[-1]) for p in result_dirs]
print(f"Found nlive={nlive_values}")
res = {}
for path, t in zip(result_dirs, nlive_values):
    files = find_results_files(path)
    res[t] = load_results(files)

In [ ]:
figsize = (5/13 * default_figsize[0], 1.8 * default_figsize[1])
fig, axs = plt.subplots(
    5, 1, sharex=True, figsize=figsize,
    gridspec_kw={"hspace": 0.1, "wspace": 0.1},
)
vals = np.array(list(res.keys()))
for nlive, r in res.items():
    axs[0].errorbar(
        nlive,
        r["final_log_evidence"].mean(),
        yerr=r["final_log_evidence"].std(),
        ls="",
        marker=".",
        c="C0",
    )


axs[0].set_ylabel(r"$\ln (\hat{Z} / Z_{\textrm{True}})$")


for nlive, r in res.items():
    axs[1].errorbar(
        nlive,
        r["final_log_evidence_error"].mean(),
        yerr=r["final_log_evidence_error"].std(),
        ls="",
        marker=".",
        c="C0",
    )

axs[1].set_ylabel(r"$\sigma [\ln \hat{Z}]$")

key = "likelihood_evaluations"

for nlive, r in res.items():
    axs[2].errorbar(
        nlive,
        r[key].mean(),
        yerr=r[key].std(),
        ls="",
        marker=".",
        c="C0",
    )
axs[2].set_ylabel("Likelihood \nevaluations")

key = "sampling_time"
for nlive, r in res.items():
    axs[3].errorbar(
        nlive,
        r[key].mean(),
        yerr=r[key].std(),
        ls="",
        marker=".",
        c="C0",
    )
axs[3].set_ylabel("Wall time [s]")

key = "ess"
for nlive, r in res.items():
    axs[4].errorbar(
        nlive,
        r[key].mean(),
        yerr=r[key].std(),
        ls="",
        marker=".",
        c="C0",
    )
axs[4].set_ylabel("ESS")

axs[-1].set_xlabel(r"$N_{\textrm{Live}}$")
# axs[-1].set_xticks(vals)

for ax in axs:
    ax.grid(axis="both")

plt.tight_layout()
# plt.subplots_adjust(hspace=0.15)

fig.savefig("figures_rerun/scaling.pdf", bbox_inches="tight")
plt.show()

In [ ]:
print("Increase in average sampling time")
print(res[10000]["sampling_time"].mean() / res[100]["sampling_time"].mean())